helper functions

In [1]:
import pickle

def load_pickle(fn):
  with open(fn, "rb") as f:
    return pickle.load(f)

# Compare both vocab files

In [16]:
MM_vocab_file = "/content/drive/MyDrive/Capstone/Deep Music Generation Group Folder/dataset/remi_vocab_MM.pkl"
our_vocab_file = "/content/drive/MyDrive/Capstone/Deep Music Generation Group Folder/dataset/remi_vocab.pkl"

In [17]:
remi_vocab = load_pickle(MM_vocab_file)
our_vocab = load_pickle(our_vocab_file)

In [18]:
len(remi_vocab[1]), len(our_vocab[1])

(332, 403)

In [ ]:
# remi_vocab[1]

sanity checks

In [19]:
# vocab sanity checks
assert len(remi_vocab[0]) == len(remi_vocab[1])
assert set(remi_vocab[0].keys()) == set(remi_vocab[1].values())
assert set(remi_vocab[0].values()) == set(remi_vocab[1].keys())
assert len(our_vocab[0]) == len(our_vocab[1])
assert set(our_vocab[0].keys()) == set(our_vocab[1].values())
assert set(our_vocab[0].values()) == set(our_vocab[1].keys())

In [ ]:
# check whether everything remi vocab has our_vocab also has
for k, v in remi_vocab[0].items():
    assert k in our_vocab[0]

# check whether for these keys, the values are the same
i = 0 
for k, v in remi_vocab[0].items():
    if v != our_vocab[0][k]:
        print(k, remi_vocab[0][k], our_vocab[0][k])
        i += 1
print(i)
print('=' * 40)

# get newly added keys
j = 0
for k, v in our_vocab[0].items():
    if k not in remi_vocab[0]:
        print(k, v)
        j += 1
print(j)

# Modify pretrained weights

In [21]:
# download MM's pretrained weights
!wget -O musemorphose_pretrained_weights.pt https://zenodo.org/record/5119525/files/musemorphose_pretrained_weights.pt?download=1

--2022-10-26 19:56:39--  https://zenodo.org/record/5119525/files/musemorphose_pretrained_weights.pt?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 359717524 (343M) [application/octet-stream]
Saving to: ‘musemorphose_pretrained_weights.pt’

musemorphose_pretra 100%[===================>] 343.05M  26.6MB/s    in 14s     

2022-10-26 19:56:55 (24.9 MB/s) - ‘musemorphose_pretrained_weights.pt’ saved [359717524/359717524]



In [24]:
import torch
pretrained_weights = torch.load("/content/musemorphose_pretrained_weights.pt")

In [26]:
len(pretrained_weights)

311

In [46]:
# map keys in our vocab to keys in the original vocab
def map_keys(remi_vocab, our_vocab):
  remi_vocab_k2idx, remi_vocab_idx2k = remi_vocab
  our_vocab_k2idx, our_vocab_idx2k = our_vocab
  ours2mm = {}
  for k, v in our_vocab_idx2k.items():
    if v in remi_vocab_k2idx:
      ours2mm[k] = remi_vocab_k2idx[v]
  return ours2mm

In [52]:
ours2mm = map_keys(remi_vocab, our_vocab)

In [78]:
def convert_mm_to_ours(pretrained, ours2mm, new_dim=404):
  our_shape = list(pretrained.shape)
  our_shape[0] = new_dim
  converted = torch.zeros(our_shape).float()
  for i in range(new_dim):
    if i in ours2mm:
      # keys existed in original vocab
      converted[i] = pretrained[ours2mm[i]]
  # add original last dim to our last dim
  converted[-1] = pretrained[-1]
  return converted

convert

In [86]:
from collections import OrderedDict

In [87]:
converted_weights = OrderedDict()
for k, v in pretrained_weights.items():
  if 333 in v.shape:   # contain vocab dim related weights
    converted_weights[k] = convert_mm_to_ours(v, ours2mm)
    print(k, converted_weights[k].shape)
  else:
    converted_weights[k] = v

token_emb.emb_lookup.weight torch.Size([404, 512])
dec_out_proj.weight torch.Size([404, 512])
dec_out_proj.bias torch.Size([404])


save results

In [88]:
torch.save(converted_weights, "converted_pretrained_weights.pt")

In [89]:
# !cp "/content/converted_pretrained_weights.pt" "/content/drive/MyDrive/Capstone/Deep Music Generation Group Folder/dataset"